## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os

In [ ]:
CHARITY_DATA_URL = "https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv"
OUTPUT_FILE_PATH = os.path.join('Output', 'AlphabetSoupCharity_Optimization.h5')

In [ ]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Define a function to replace values below a certain threshold
def replace_values(data_series, threshold):
    doomed_value_series =data_series.value_counts() < threshold
    doomed_values = data_series.value_counts()[doomed_value_series].index

    replacement_fs = data_series.isin(doomed_values)
    data_series[replacement_fs] = 'Other'
    return data_series

In [ ]:
# # Call the replace_values function to replace the values in the Application_Type column, with a cutoff of 500
application_df['APPLICATION_TYPE'] = replace_values(application_df['APPLICATION_TYPE'], 500)

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Call the replace_values function to replace the values in the Classification column, with a cutoff of 1000
application_df['CLASSIFICATION'] = replace_values(application_df['CLASSIFICATION'], 1000)
    
# # Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [ ]:
# Split our preprocessed data into our features and target arrays
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=3, activation="sigmoid", input_dim=43))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# # Third hidden layer
# nn.add(tf.keras.layers.Dense(units=9, activation="tanh"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=15)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save(OUTPUT_FILE_PATH, save_format='h5')